In [1]:
import sys
sys.path.insert(0,'../scripts')

import warnings

warnings.filterwarnings("ignore")
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from torchvision.datasets import ImageFolder

import prepare_datasets
from main import parse_args
import helper

In [2]:
def to_numpy(t):
    array = torch.clip((t * 0.224) + 0.45, 0, 1).permute(1, 2, 0).numpy()
    return array

In [3]:
args = parse_args(params_path='/home/yue3/DINO/scripts/yaml/ViT-S-16-DDSM.yaml')
augmentation_params = args['augmentation_params']
dataset_params = args['dataset_params']
transforms_aug = prepare_datasets.DataAugmentationDINO(dataset_params['augmentations'], augmentation_params['global_crops_scale'],
        augmentation_params['local_crops_scale'], augmentation_params['local_crops_number'],
        augmentation_params['full_size'], augmentation_params['global_size'], augmentation_params['local_size'])
transforms_plain_for_lineartrain_ddsm = transforms_aug.transforms_plain_for_lineartrain_ddsm
transforms_plain_for_lineartrain = transforms_aug.transforms_plain_for_lineartrain
dataset_name = dataset_params['dataset_choice']['dataset_name']
dataset_class = prepare_datasets.GetDatasets(data_folder=dataset_params['data_folder'],
                                                 dataset_name=dataset_name,
                                                 knn_use_cuda=dataset_params['dataset_choice'][dataset_name]['knn_use_cuda'],
                                                 label_mapping_path=dataset_params['dataset_choice'][dataset_name]['label_mapping_path'])
if dataset_name == 'CBISDDSM':
    train_plain_for_lineartrain_dataset = dataset_class.get_datasets('train/', transforms_plain_for_lineartrain_ddsm, include_index=False)
else:
    train_plain_for_lineartrain_dataset = dataset_class.get_datasets('train/', transforms_plain_for_lineartrain, include_index=False)






Model is going to be save in  /storage/yue/dino_models/vit_small_CBISDDSM_finetuning_official



ARGS ARE: 
{
    "start_training": {
        "mode": "train_finetuning",
        "eval": {
            "choices": [
                "if_knn"
            ],
            "epoch": 99,
            "linear": {
                "batch_size": 1024,
                "num_epochs": 100,
                "n_last_blocks": 4,
                "avgpool_patchtokens": false,
                "lr": 0.001,
                "momentum": 0.9,
                "wd": 0,
                "val_freq": 1
            }
        },
        "train_finetuning": {
            "ckp_path_choice": "Official",
            "ckp_path": {
                "Official": "/storage/yue/dino_models/dino_deitsmall16_pretrain_full_checkpoint_official.pth",
                "Ours": "/storage/yue/dino_models/vit_small_ImageNet/checkpoint0099.pth"
            },
            "batch_size": 128,
            "num_epochs": 100,
            "n_last_bloc

In [4]:
@ipywidgets.interact
def _(
    i=ipywidgets.IntSlider(min=0, max=len(train_plain_for_lineartrain_dataset) - 1, continuous_update=False),
    seed=ipywidgets.IntSlider(min=0, max=50, continuous_update=False),
):
    torch.manual_seed(seed)
    
    original_img = train_plain_for_lineartrain_dataset[i][0]
    print(original_img.shape)
    _, ax_orig = plt.subplots(figsize=(15, 5))
    ax_orig.imshow(to_numpy(original_img))
    ax_orig.set_title("Original")
    ax_orig.axis("off")
    

interactive(children=(IntSlider(value=0, continuous_update=False, description='i', max=2418), IntSlider(value=…